In [17]:
param_lgb = {'subsample': 0.95,
             'colsample_bytree': 0.9,
             'max_depth': 50,
             'min_child_weight': 0.0029805017044362268,
             'min_child_samples': 10,
             'num_leaves': 381, # use 381
             'reg_alpha': 1.0,
             'reg_lambda': 2.0,
             'n_estimators':1000,
             'learning_rate': 0.03,
             'early_stopping_rounds': 100,
             'first_metric_only': True,
             #'class_weight':'balanced',
             
                'objective': 'binary',
                'save_binary': True,
                'seed': 1337,
                'feature_fraction_seed': 1337,
                'bagging_seed': 1337,
                'drop_seed': 1337,
                'data_random_seed': 1337,
                'boosting_type': 'gbdt',
                'verbose': 100,
                'is_unbalance': False,
                'boost_from_average': True,
                'metric':'auc'
    }

In [28]:
def CV_train(train_df, features):
    plt.rcParams["axes.grid"] = True
    nfold = 5
    #skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=42)
    skf = KFold(n_splits=nfold, shuffle=False, random_state=42)

    oof = np.zeros(len(train_df))
    mean_fpr = np.linspace(0,1,100)
    cms= []
    tprs = []
    aucs = []
    y_real = []
    y_proba = []
    recalls = []
    roc_aucs = []
    f1_scores = []
    accuracies = []
    precisions = []
    #predictions = np.zeros(len(test_df))
    feature_importance_df = pd.DataFrame()
    
    print('feature length', len(features))

    i = 1
    clf = lgb.LGBMClassifier()
    clf.set_params(**param_lgb)
    
    for train_idx, valid_idx in skf.split(train_df, train_df[target].values):
        gc.collect()
        x_train = train_df.iloc[train_idx][features]
        y_train = train_df.iloc[train_idx][target]

        x_val = train_df.iloc[valid_idx][features]
        y_val = train_df.iloc[valid_idx][target]


        clf.fit(x_train, y_train,   
                eval_metric =['auc'],
                verbose = 100,
                eval_set = (x_val, y_val) )  



        roc_aucs.append(roc_auc_score(y_val.values,
                                      clf.predict_proba(x_val)[:,1] ))



    print(
        '\nCV roc score        : {0:.4f}, std: {1:.4f}.'.format(np.mean(roc_aucs), np.std(roc_aucs))
        )


    return np.mean(roc_aucs)

def automated_add_cv(train_df, 
                     test_df,
                     base_score,
                     features, 
                     feature_to_add, 
                     param_lgb):
    '''
    - train_df, test_df: data used
    - features: baseline features
    - base_score: baseline score
    - feature_to_add: features to be added one by one and test if CV score will improve or not 
    - param_lgb: hyper parameter used for lgb
    '''
    
    history = []
    base_score = deepcopy(base_score)
    
    for fea in feature_to_add:
        print('score before', fea, base_score)
        start = time()       
        
        features_tmp = deepcopy(features)
        features_tmp.append(fea)
        print('add:', fea, 'Total:', len(features_tmp), 'correct?:', fea in features_tmp)
            
        score = CV_train(train_df, features_tmp)
        history.append(score)
        gc.collect()
              
        if score >= base_score:
            features = deepcopy(features_tmp)
            base_score = score
         
        print(fea, 'score:', score)    
        print('score after', fea, base_score, 'add?', fea in features)
        print(fea, 'excution time', time() - start)
            
    return (features, history)

def automated_remove_cv(train_df, 
                     base_score,
                     features, 
                     feature_to_remove, 
                     param_lgb):
    
    history = []
    base_score = deepcopy(base_score)
    
    for fea in feature_to_remove:
        print('score before', fea, base_score)
        start = time()       
        
        features_tmp = deepcopy(features)
        del features_tmp[features_tmp.index(fea)]
        print('remove:', fea, 'Total:', len(features_tmp), 'correct?:', fea not in features_tmp)
            
        score = CV_train(train_df, features_tmp)
        history.append(score)
        gc.collect()
        
        print(fea, 'score', score)
        if score >= base_score:
            features = deepcopy(features_tmp)
            base_score = score
         
            
        print('score after', fea, base_score, 'add?', fea in features)
        print(fea, 'excution time', time() - start)
            
    return (features, history)

#### Explaination
- train_df, test_df: data used
- features: baseline features
- base_score: baseline score
- feature_to_add: features to be added one by one and test if CV score will improve or not 
- param_lgb: hyper parameter used for lgb

In [ ]:
result = automated_add_cv(train_df = train_df, 
                          test_df = test_df, 
                          base_score = base_score,
                          features = features, 
                          feature_to_add = fea_importance, 
                          param_lgb = param_lgb)

score before DateC9minus_mean_all 0.9363510457876616
add: DateC9minus_mean_all Total: 336 correct?: True
feature length 336
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.904927
[200]	valid_0's auc: 0.918914
[300]	valid_0's auc: 0.922871
[400]	valid_0's auc: 0.924732
[500]	valid_0's auc: 0.925103
[600]	valid_0's auc: 0.92518
Early stopping, best iteration is:
[537]	valid_0's auc: 0.925364
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.925386
[200]	valid_0's auc: 0.938147
[300]	valid_0's auc: 0.940973
[400]	valid_0's auc: 0.941878
[500]	valid_0's auc: 0.942272
[600]	valid_0's auc: 0.94234
Early stopping, best iteration is:
[528]	valid_0's auc: 0.942488
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.922247
[200]	valid_0's auc: 0.934216
[300]	valid_0's auc: 0.935941
[400]	valid_0's auc: 0.935997
[500]	valid_0's auc: 0.935954
Early stopping, best iteration is:
[417]	valid_0'

[500]	valid_0's auc: 0.9241
Early stopping, best iteration is:
[438]	valid_0's auc: 0.924302
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.924329
[200]	valid_0's auc: 0.936959
[300]	valid_0's auc: 0.940194
[400]	valid_0's auc: 0.941698
[500]	valid_0's auc: 0.941883
[600]	valid_0's auc: 0.941862
[700]	valid_0's auc: 0.941995
Early stopping, best iteration is:
[689]	valid_0's auc: 0.942018
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.923892
[200]	valid_0's auc: 0.934734


[300]	valid_0's auc: 0.936455
[400]	valid_0's auc: 0.936441
Early stopping, best iteration is:
[359]	valid_0's auc: 0.936651
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.940746
[200]	valid_0's auc: 0.950855
[300]	valid_0's auc: 0.952382
[400]	valid_0's auc: 0.952726
Early stopping, best iteration is:
[384]	valid_0's auc: 0.952872
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.919286
[200]	valid_0's auc: 0.931145
[300]	valid_0's auc: 0.932216
[400]	valid_0's auc: 0.932619
[500]	valid_0's auc: 0.932489
[600]	valid_0's auc: 0.932411
Early stopping, best iteration is:
[538]	valid_0's auc: 0.932794

CV roc score        : 0.9377, std: 0.0095.
Date+HourD10minus_mean_all score: 0.9377253212534289
score after Date+HourD10minus_mean_all 0.9377253212534289 add? True
Date+HourD10minus_mean_all excution time 820.4176063537598
score before DateC13minus_mean_all 0.9377253212534289
add: DateC13minus_mean_all Total: 338 cor

[100]	valid_0's auc: 0.903551
[200]	valid_0's auc: 0.917713
[300]	valid_0's auc: 0.921894
[400]	valid_0's auc: 0.923031
[500]	valid_0's auc: 0.923512
[600]	valid_0's auc: 0.923627
[700]	valid_0's auc: 0.924499
[800]	valid_0's auc: 0.924556
Early stopping, best iteration is:
[751]	valid_0's auc: 0.924739
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.925725
[200]	valid_0's auc: 0.938017
[300]	valid_0's auc: 0.940379
[400]	valid_0's auc: 0.941164
[500]	valid_0's auc: 0.941667
[600]	valid_0's auc: 0.941573
Early stopping, best iteration is:
[535]	valid_0's auc: 0.941859
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.922896
[200]	valid_0's auc: 0.934189
[300]	valid_0's auc: 0.936532
[400]	valid_0's auc: 0.936788
[500]	valid_0's auc: 0.936463
Early stopping, best iteration is:
[425]	valid_0's auc: 0.936834
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.940382
[200]	valid_0's 

Early stopping, best iteration is:
[378]	valid_0's auc: 0.952022
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.918606
[200]	valid_0's auc: 0.929101
[300]	valid_0's auc: 0.930941
[400]	valid_0's auc: 0.931214
Early stopping, best iteration is:
[378]	valid_0's auc: 0.931389

CV roc score        : 0.9374, std: 0.0093.
Date+HourD1minus_mean_all score: 0.9374179359944332
score after Date+HourD1minus_mean_all 0.9380838517460729 add? False
Date+HourD1minus_mean_all excution time 819.3400919437408
score before DateD1minus_mean_all 0.9380838517460729
add: DateD1minus_mean_all Total: 339 correct?: True
feature length 339
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.905353
[200]	valid_0's auc: 0.920137
[300]	valid_0's auc: 0.923872
[400]	valid_0's auc: 0.924985
[500]	valid_0's auc: 0.925657
Early stopping, best iteration is:
[490]	valid_0's auc: 0.925782
Training until validation scores don't improve for 100 rounds.


...

[100]	valid_0's auc: 0.925043
[200]	valid_0's auc: 0.93769
[300]	valid_0's auc: 0.940475
[400]	valid_0's auc: 0.941339
Early stopping, best iteration is:
[399]	valid_0's auc: 0.941404
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.92299
[200]	valid_0's auc: 0.933878
[300]	valid_0's auc: 0.935829
[400]	valid_0's auc: 0.936247
[500]	valid_0's auc: 0.936219
Early stopping, best iteration is:
[421]	valid_0's auc: 0.936348
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.940321
[200]	valid_0's auc: 0.950506
[300]	valid_0's auc: 0.952632
[400]	valid_0's auc: 0.952705
Early stopping, best iteration is:
[384]	valid_0's auc: 0.95278
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.918647
[200]	valid_0's auc: 0.930642
[300]	valid_0's auc: 0.931915
[400]	valid_0's auc: 0.931834
Early stopping, best iteration is:
[352]	valid_0's auc: 0.932102

CV roc score        : 0.9373, std: 0.0096.


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.904141
[200]	valid_0's auc: 0.918294
[300]	valid_0's auc: 0.922087
[400]	valid_0's auc: 0.922818
[500]	valid_0's auc: 0.922853
[600]	valid_0's auc: 0.92364
[700]	valid_0's auc: 0.923813
[800]	valid_0's auc: 0.923942
Early stopping, best iteration is:
[740]	valid_0's auc: 0.924189
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.924689
[200]	valid_0's auc: 0.937695
[300]	valid_0's auc: 0.94027
[400]	valid_0's auc: 0.941497
[500]	valid_0's auc: 0.941644
[600]	valid_0's auc: 0.941955
[700]	valid_0's auc: 0.941723
Early stopping, best iteration is:
[652]	valid_0's auc: 0.942074
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.923569
[200]	valid_0's auc: 0.934303
[300]	valid_0's auc: 0.935618
[400]	valid_0's auc: 0.936176
[500]	valid_0's auc: 0.935911
Early stopping, best iteration is:
[420]	valid_0's auc: 0.936206
Training until val

[400]	valid_0's auc: 0.951486
[500]	valid_0's auc: 0.951301
Early stopping, best iteration is:
[440]	valid_0's auc: 0.951642
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.917453
[200]	valid_0's auc: 0.929559
[300]	valid_0's auc: 0.931403
[400]	valid_0's auc: 0.93188
[500]	valid_0's auc: 0.931649
Early stopping, best iteration is:
[414]	valid_0's auc: 0.931962

CV roc score        : 0.9375, std: 0.0092.
dayofwweekTransactionAmtminus_mean_all score: 0.9374821688791997
score after dayofwweekTransactionAmtminus_mean_all 0.9380838517460729 add? False
dayofwweekTransactionAmtminus_mean_all excution time 806.5990867614746
score before dayofwweekae_reconstruction_errorminus_mean_all 0.9380838517460729
add: dayofwweekae_reconstruction_errorminus_mean_all Total: 339 correct?: True
feature length 339
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.904391
[200]	valid_0's auc: 0.919463
[300]	valid_0's auc: 0.923951
[400]	

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.904655
[200]	valid_0's auc: 0.919096
[300]	valid_0's auc: 0.923581
[400]	valid_0's auc: 0.924678
[500]	valid_0's auc: 0.924666
[600]	valid_0's auc: 0.925162
[700]	valid_0's auc: 0.925588
Early stopping, best iteration is:
[679]	valid_0's auc: 0.925725
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.92388
[200]	valid_0's auc: 0.937902
[300]	valid_0's auc: 0.941056
[400]	valid_0's auc: 0.942394
[500]	valid_0's auc: 0.942708
[600]	valid_0's auc: 0.942973
[700]	valid_0's auc: 0.942746
Early stopping, best iteration is:
[607]	valid_0's auc: 0.943025
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.92213
[200]	valid_0's auc: 0.934553
[300]	valid_0's auc: 0.936025
[400]	valid_0's auc: 0.936701
[500]	valid_0's auc: 0.93677
Early stopping, best iteration is:
[414]	valid_0's auc: 0.936817
Training until validation scores don't improve fo

[100]	valid_0's auc: 0.917908
[200]	valid_0's auc: 0.929368
[300]	valid_0's auc: 0.930931
[400]	valid_0's auc: 0.931128
Early stopping, best iteration is:
[362]	valid_0's auc: 0.93128

CV roc score        : 0.9378, std: 0.0093.
Date+HourC14minus_mean_all score: 0.9378355589505185
score after Date+HourC14minus_mean_all 0.9381610563355001 add? False
Date+HourC14minus_mean_all excution time 885.7371912002563
score before DateC4minus_mean_all 0.9381610563355001
add: DateC4minus_mean_all Total: 340 correct?: True
feature length 340
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.903368
[200]	valid_0's auc: 0.917359
[300]	valid_0's auc: 0.920515
[400]	valid_0's auc: 0.920719
[500]	valid_0's auc: 0.921146
[600]	valid_0's auc: 0.921126
Early stopping, best iteration is:
[536]	valid_0's auc: 0.921372
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.924282
[200]	valid_0's auc: 0.937885
[300]	valid_0's auc: 0.941383
[400]	